In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_friedman1
from sklearn.metrics import mean_squared_error
import sklearn.model_selection as ms

In [2]:
import numpy as np
np.random.seed(123)
import collections, copy, pickle
from termcolor import colored
from importlib import reload
from dateutil.parser import parse
import scipy.linalg, scipy.stats
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
import seaborn as sns
from IPython.display import HTML

In [3]:
from mlxtend.frequent_patterns import apriori

import sklearn.ensemble
import sklearn.svm
import sklearn.tree
import sklearn.linear_model
import sklearn.neighbors

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [88]:
import util.plot
import util.data

In [89]:
reload(util.plot)
reload(util.data);

In [90]:
with open('data/encoder.pkl', 'rb') as f:
    E = pickle.load(f)

In [91]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=10*1000)
data_test = pd.read_csv('data/test_set_VU_DM.csv', sep=',', nrows=1000)

In [92]:
for k in data.columns:
    if data[k].isna().sum() > 0:
        print('rm ', k)
        data.drop(columns=[k], inplace=True)

varsUsed = list(data.columns)
# varsUsed.remove('Unnamed')
varsUsed.remove('position')
varsUsed.remove('random_bool')
len(varsUsed)

rm  orig_destination_distance
rm  comp1_rate
rm  comp1_inv
rm  comp1_rate_percent_diff
rm  comp2_rate
rm  comp2_inv
rm  comp2_rate_percent_diff
rm  comp3_rate
rm  comp3_inv
rm  comp3_rate_percent_diff
rm  comp4_rate
rm  comp4_inv
rm  comp4_rate_percent_diff
rm  comp5_rate
rm  comp5_inv
rm  comp5_rate_percent_diff
rm  comp6_rate
rm  comp6_inv
rm  comp6_rate_percent_diff
rm  comp7_rate
rm  comp7_inv
rm  comp7_rate_percent_diff
rm  comp8_rate
rm  comp8_inv
rm  comp8_rate_percent_diff


48

In [93]:
E.discretizers

{'srch_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'site_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'visitor_location_country_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'prop_country_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'prop_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'srch_destination_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'srch_adults_count': KBinsDiscretizer(encode='ordinal', n_bins=array([5]), strategy='uniform'),
 'srch_children_count': KBinsDiscretizer(encode='ordinal', n_bins=array([3]), strategy='uniform'),
 'srch_room_count': KBinsDiscretizer(encode='ordinal', n_bins=array([2]), strategy='uniform'),
 'position': KBinsDiscretizer(encode='ordinal', n_bins=array([38]), strategy='uniform'),
 'srch_length_of_stay': KBinsDiscretizer(encode='ordinal',

# Collaborative Filtering (CF)

Neglect user and item info, and focus on the past behaviour, i.e. the ratings of user-item pairs.
How to predict a new value?

Firstly, the user-item (user-hotel) matrix $M_{ij}$ can be normalized by subtracting the mean (row) rating of each rating of that user.
$$ \forall \langle i,j\rangle \quad M_{ij} = \begin{cases}
M_{ij} - \sum_j M_{ij} & \text{if exists} \\
0 & \text{otherwise} 
\end{cases}
$$

(as done in centerd cosine similarity)


### 1) Encode missing data, i.e. the hotels with no clicks

- KNN
    - Impute missing values using similar records (KNN, regression)
- Pearson correlation to measure similarity

### 2) Item-based similarity

#### User-user similarity

When predicting the score w.r.t. a user, average over similar users

- Jaccard similarity - assumes boolean fields
$$\text{sim}(a,b) = \frac{| r_a \cup r_b |}{| r_a \cap r_b |}$$

- Cosine similarity (n-dimensional)
$$\text{sim}(a,b) = \cos(r_a,r_b)$$
    - problem: penalizes missing values

- Centred Cosine similarity (n-dimensional) i.e. pearson correlation
    - Normalize ratings by subtrating the average user rating: $r_{\mu}$
    - $r_a = r_a - r_{\mu}$
    - Missing values are treated as average and critical users are taken with a grain of salt


Then the "average" rating $r_{x,i}$ of a group of $k$ users (similar to user $x$) for item $i$ is:
$r_{x,i} = \frac1k \sum_{j=1}^k r_{j,i}$

Incorporating the similarity measure (distance function)
$$ r_{x,i} = \frac1{\sum_j \text{sim}(x,j)} \sum_{j=1}^k \text{sim}(x,j)\, r_{j,i} $$


#### Item-item similarity

Intuitively, items are often more similar than people. It is often easier to predict the quantitative similarity (distance) between items than people.
Items can even be equivalent (e.g. two hotels next to each other with the same specs), whereas people are very unlikely to be equivalent.


The average rating $r_{x,i}$ of a group of $k$ items (similar to item $i$) for (possibly average) user $x$ is:
$r_{x,i} = \frac1k \sum_{j=1}^k r_{x,j}$

Incorporating a similarity measure (distance function)
$$ r_{x,i} = \frac1{\sum_{j=1}^k \text{sim}(x,j)} \sum_{j=1}^k \text{sim}(x,j)\, r_{x,j} $$



### 3) SVD

The aforementioned approaches use dense matrices that don't scale very well. Using a _latent factor model_ the sparsity of the data can be used to our advantage. SVD works as a kind of dimensionaliy reduction (i.e. compressing the data into the most important features).
Note that the matrix still has to be subtracted by the row-means.


For $m$ users, $n$ items (hotels), $M: m \times n$, $U: m\times m$, $\Sigma: m\times n$, $V^T n \times n$.

$\Sigma$ is a diagonal with singular values.

$$ M = U \Sigma V^T$$
i.e.
$$ M_k = U_k \Sigma_k V_k^T$$


In [94]:
# find USV by gradient descent, ignoring the missing terms?

In [95]:
A = scipy.sparse.csc_matrix([[1, 0, 0], [5, 0, 2], [0, -1, 0], [0, 0, 3]], dtype=float)
u, s, vt = scipy.sparse.linalg.svds(A, k=2) # k is the number of factors
A.shape, A.toarray()

((4, 3), array([[ 1.,  0.,  0.],
        [ 5.,  0.,  2.],
        [ 0., -1.,  0.],
        [ 0.,  0.,  3.]]))

In [96]:
sI = scipy.sparse.csc_matrix(np.diagflat(s))
sI = np.diagflat(s)

In [97]:
A_pred = np.matmul(u, np.matmul(sI,vt))
np.round(A_pred,4)
# u.shape, s.shape, vt.shape, sI.shape

array([[ 1.,  0.,  0.],
       [ 5., -0.,  2.],
       [-0., -0.,  0.],
       [ 0., -0.,  3.]])

In [98]:
# ratingsMatrix[user][movie] = sum(userFeature[f][user] * movieFeature[f][movie]) for f from 1 to 40
# for k features, n users, m items
# B: n x m x k
# U: n x k     user preferences
# S: k x k     or   k
# Vt: k x m    movie aspects

In [103]:
scores = {'item':[], 'user': [], 'score':[]}
for i, srch_id in enumerate(data.srch_id[:10]):
    row = data.iloc[i]
    scores['item'].append(row.srch_id)
    scores['user'].append(row.srch_id)
    scores['score'].append(row.score)

srch_id                               8.000000
site_id                               0.000000
visitor_location_country_id           8.000000
visitor_hist_starrating              -0.961621
visitor_hist_adr_usd                  0.178773
prop_country_id                       0.000000
prop_id                               0.000000
prop_starrating                      -0.216502
prop_review_score                    -0.209344
prop_brand_bool                       1.000000
prop_location_score1                 -0.149001
prop_location_score2                 -0.615103
prop_log_historical_price             1.000000
position                             26.000000
price_usd                            -0.124258
promotion_flag                        0.000000
srch_destination_id                   8.000000
srch_length_of_stay                   0.000000
srch_booking_window                   0.000000
srch_adults_count                     3.000000
srch_children_count                   0.000000
srch_room_cou

In [104]:
data.head()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,hour,minute,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,score
0,8.0,0.0,8.0,-0.961621,0.178773,0.0,0.0,-0.216502,-0.209344,1,...,8,32,0,0,0,0,1,0,0,0
1,8.0,0.0,8.0,-0.961621,0.178773,0.0,0.0,0.814459,0.252784,1,...,8,32,0,0,0,0,1,0,0,0
2,8.0,0.0,8.0,-0.961621,0.178773,0.0,0.0,-0.216502,0.714912,1,...,8,32,0,0,0,0,1,0,0,0
3,8.0,0.0,8.0,-0.961621,0.178773,0.0,0.0,-1.247462,0.252784,1,...,8,32,0,0,0,0,1,0,0,0
4,8.0,0.0,8.0,-0.961621,0.178773,0.0,8.0,0.814459,-0.209344,1,...,8,32,0,0,0,0,1,0,0,0


In [85]:
ratings = {'itemID': [1, 1, 1, 2, 2],
            'userID': [9, 32, 2, 45, 'user_foo'],
            'rating': [3, 2, 4, 3, 1]}
tuples = pd.DataFrame(ratings)
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(tuples, reader)
cross_validate(NormalPredictor(), data, cv=2)

{'test_rmse': array([1.76206528, 2.03850927]),
 'test_mae': array([1.71642025, 1.50713904]),
 'fit_time': (0.002329111099243164, 6.771087646484375e-05),
 'test_time': (0.0017549991607666016, 5.3882598876953125e-05)}

In [ ]:

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate


# Creation of the dataframe. Column names are irrelevant.
ratings_dict = {'itemID': [1, 1, 1, 2, 2],
                'userID': [9, 32, 2, 45, 'user_foo'],
                'rating': [3, 2, 4, 3, 1]}
df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)

In [84]:
# https://surprise.readthedocs.io/en/stable/getting_started.html#load-from-df-example
from surprise import Reader, Dataset, SVD, NormalPredictor
from surprise import evaluate
from surprise.model_selection import cross_validate
# algo = SVD()
# evaluate(algo, data, measures=['RMSE', 'MAE'])

In [76]:
fn = 'data/tuples.csv'
reader = Reader(sep=';', rating_scale=(0.,1.))
tuples = Dataset.load_from_df

In [77]:
tuples.

In [105]:
# note that even though the training score is an integer,
# the predicted score is a float. The remainder terms contain information about confidence.

y_labels = ['click_bool', 'booking_bool', 'gross_bookings_usd', 'score']
y = data['score']
x = data[varsUsed].drop(columns=y_labels)
# x.shape, y.shape
# x_train, x_test, y_train, y_test = ms.train_test_split(x, y, test_size=0.3, random_state=42)
# pd.DataFrame.drop()
xy_train, xy_test = ms.train_test_split(data, random_state=42, test_size=0.3)
y = 'score'
y_train = xy_train[y].values
y_test = xy_test[y].values
x_train = xy_train.drop(columns=[y]).values
x_test = xy_test.drop(columns=[y]).values

# y_train = y_train[:,np.newaxis]
# y_test = y_test[:,np.newaxis]

In [109]:
def scores_df(data):
    scores = {'item':[], 'user': [], 'score':[]}
    for i in range(x.shape[0]):
        row = data.iloc[i]
        scores['item'].append(row.srch_id)
        scores['user'].append(row.prop_id)
        scores['score'].append(row.score)
    return pd.DataFrame(scores)

In [113]:
scores = scores_df(data)

In [11]:
def cross_validation(model_func, x_train, y_train, k=None, results=None, v=0):
    # Train for 5 folds, returing ROC AUC. You can also try 'accuracy' as a scorer
    n_folds = 5
    # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    scoring1 = 'explained_variance' # explained_variance neg_mean_squared_error r2
    scores_1 = cross_val_score(model_func, x_train, y_train, cv=n_folds, scoring=scoring1)
    scoring2 = 'r2' # explained_variance neg_mean_squared_error r2
    scores_2 = cross_val_score(model_func, x_train, y_train, cv=n_folds, scoring=scoring2)
    
    if results is not None:
        results[k] = (scores_1, scores_2)
    if v:
        print('scores per fold ', scores_acc)
        print('  mean score    ', np.mean(scores_acc))
        print('  standard dev. ', np.std(scores_acc))
    return (scoring1, scoring2)

In [12]:
kwargs = {'random_state': 123}
models = {'GBoost 2': GradientBoostingRegressor(n_estimators=1000, learning_rate=0.001,
                                max_depth=2, loss='ls', **kwargs),
          'GBoost 4': GradientBoostingRegressor(n_estimators=100, learning_rate=0.01,
                                max_depth=4, loss='ls', **kwargs),
          'RForest 2': RandomForestClassifier(n_estimators=1000, max_depth=2, **kwargs),
          'RForest 4': RandomForestClassifier(n_estimators=1000, max_depth=4, **kwargs),
          'Logit': sklearn.linear_model.LogisticRegression(solver='liblinear', multi_class='ovr', **kwargs),
          'SVR': sklearn.svm.SVR(kernel='linear'),
          'KNN 5': sklearn.neighbors.KNeighborsClassifier(n_neighbors=5),
          'Ensemble Bagging': sklearn.ensemble.BaggingClassifier(n_estimators=100, **kwargs),
         }

results = {}
for k,m in models.items():
    print(k)
    scoring1, scoring2 = cross_validation(m, x_train, y_train, k, results)
#     cross_val_score(m, x_train, y_train, cv=n_folds, v=1)
print('\nscoring:', scoring1, scoring2)

GBoost 2
GBoost 4
RForest 2
RForest 4
Logit
SVR
KNN 5
Ensemble Bagging

scoring: explained_variance r2


In [13]:
print('Model & Mean score1 & Std score1 & Mean score2 & Std score2 \\\\ \n\\hline')
best_k = ''
best_mean = 0
for k, (scores_acc, scores_2) in results.items():
    if np.mean(scores_acc) > best_mean:
        best_mean = np.mean(scores_acc)
        best_k = k
    print('%s & %0.4f & %0.4f & %0.4f & %0.4f\\\\' % (k, np.mean(scores_acc), np.std(scores_acc), np.mean(scores_2), np.std(scores_2)))
print('\nbest acc:', best_k, round(best_mean,4))

Model & Mean score1 & Std score1 & Mean score2 & Std score2 \\ 
\hline
GBoost 2 & 0.8648 & 0.0000 & 0.8642 & 0.0009\\
GBoost 4 & 0.8660 & 0.0000 & 0.8655 & 0.0009\\
RForest 2 & 0.4573 & 0.3548 & 0.4439 & 0.3658\\
RForest 4 & 0.9883 & 0.0029 & 0.9882 & 0.0030\\
Logit & 0.5755 & 0.2589 & 0.5671 & 0.2653\\
SVR & 0.4333 & 0.2321 & 0.4190 & 0.2510\\
KNN 5 & 0.0000 & 0.0000 & -0.0317 & 0.0036\\
Ensemble Bagging & 1.0000 & 0.0000 & 1.0000 & 0.0000\\

best acc: Ensemble Bagging 1.0


In [14]:
best_k = ''
best_mean = 0
for k, (_, scores_2) in results.items():
    if np.mean(scores_2) > best_mean:
        best_mean = np.mean(scores_2)
        best_k = k
print('\nbest score 2:', best_k, round(best_mean,4))


best score 2: Ensemble Bagging 1.0


In [15]:
# performance on test data
for k, est in models.items():
    est.fit(x_train, y_train)
    print(k,':', mean_squared_error(y_test, est.predict(x_test)))

GBoost 2 : 0.12870701795983902
GBoost 4 : 0.1275453693298852
RForest 2 : 0.26666666666666666
RForest 4 : 0.02666666666666667
Logit : 0.10333333333333333
SVR : 0.5285970741544384
KNN 5 : 0.9866666666666667
Ensemble Bagging : 0.0


In [16]:
# performance on training data
for k, est in models.items():
    est.fit(x_train, y_train)
    print(k,':', mean_squared_error(y_train, est.predict(x_train)))

GBoost 2 : 0.1295480166794545
GBoost 4 : 0.1283787775930833
RForest 2 : 0.11428571428571428
RForest 4 : 0.008571428571428572
Logit : 0.002857142857142857
SVR : 0.4532242436615893
KNN 5 : 0.9885714285714285
Ensemble Bagging : 0.0


## Grid search to find best params

In [17]:
0.0001, 1e-4

(0.0001, 0.0001)

In [18]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html#sphx-glr-auto-examples-model-selection-plot-grid-search-digits-py
kwargs = {'random_state': 123}
params = {}
models = {}
models['GBoost'] = GradientBoostingRegressor(criterion='friedman_mse', loss='ls', tol=1e-4, **kwargs)
params['GBoost'] = {'n_estimators': [100,1000], 
                    'learning_rate': [0.1, 0.01, 0.001],
                    'max_depth': [2,3,4]
                   }
models['Random Forest'] = RandomForestClassifier(criterion='gini', n_jobs=2, **kwargs)
params['Random Forest'] = {'n_estimators': [100,1000], 
                            'max_depth': [2,3,4]
                           }
models['Logit'] = sklearn.linear_model.LogisticRegression(solver='sag', # liblinear, multi_class='ovr'
                            multi_class='multinomial', tol=1e-4, n_jobs=2, **kwargs)
params['Logit'] = {'C': [1, ]}
models['SVR'] = sklearn.svm.SVR(gamma='scale', epsilon=0.1 ,tol=1e-3) # gamma = scale: 1 / n_features * X.var()
params['SVR'] = {'kernel': ['rbf','linear'], 'C': [1, 10, 100]}


scores = ['explained_variance'] # explained_variance neg_mean_squared_error r2

for k, model in models.items():
    for score in scores:
        print("\n# %s (score: %s)" % (k, score))
        assert k in params.keys(), 'models and params should have the same keys'
        clf = sklearn.model_selection.GridSearchCV(model, params[k], cv=5, scoring=score)
        clf.fit(x_train, y_train)
        print("Best params (train)")
        print('\t', clf.best_params_)
        y_true, y_pred = y_test, clf.predict(x_test)
        mse = mean_squared_error(y_true, y_pred)
        print(colored('mse: %0.4f' % mse, 'green'))


# GBoost (score: explained_variance)
Best params (train)
	 {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}
mse: 0.0000

# Random Forest (score: explained_variance)


/Users/mark/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best params (train)
	 {'max_depth': 3, 'n_estimators': 100}
mse: 0.0267

# Logit (score: explained_variance)


/Users/mark/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Best params (train)
	 {'C': 1}
mse: 0.9867

# SVR (score: explained_variance)
Best params (train)
	 {'C': 1, 'kernel': 'linear'}
mse: 0.5286
